In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [3]:
def multiscale_feature_learning(inputs):
    """
    Parallel convolutions with different kernel sizes for multi-scale feature extraction
    """
    conv1 = layers.Conv2D(32, kernel_size=1, activation="relu", padding="same")(inputs)
    conv3 = layers.Conv2D(32, kernel_size=3, activation="relu", padding="same")(inputs)
    conv5 = layers.Conv2D(32, kernel_size=5, activation="relu", padding="same")(inputs)
    return layers.Concatenate()([conv1, conv3, conv5])

In [4]:
class AttentionModule(layers.Layer):
    """
    Self-attention mechanism for feature refinement
    """
    def __init__(self, **kwargs):
        super(AttentionModule, self).__init__(**kwargs)

    def build(self, input_shape):
        self.channels = input_shape[-1]
        self.query_conv = layers.Conv2D(self.channels // 8, kernel_size=1)
        self.key_conv = layers.Conv2D(self.channels // 8, kernel_size=1)
        self.value_conv = layers.Conv2D(self.channels, kernel_size=1)
        super(AttentionModule, self).build(input_shape)

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        height, width = inputs.shape[1], inputs.shape[2]

        # Query, Key, Value projections
        query = self.query_conv(inputs)
        key = self.key_conv(inputs)
        value = self.value_conv(inputs)

        # Reshape for attention computation
        query = tf.reshape(query, [batch_size, -1, self.channels // 8])
        key = tf.reshape(key, [batch_size, -1, self.channels // 8])
        value = tf.reshape(value, [batch_size, -1, self.channels])

        # Compute attention scores
        attention_weights = tf.matmul(query, key, transpose_b=True)
        attention_weights = tf.nn.softmax(attention_weights / tf.sqrt(tf.cast(self.channels // 8, tf.float32)))

        # Apply attention to values
        attention_output = tf.matmul(attention_weights, value)
        return tf.reshape(attention_output, [batch_size, height, width, self.channels])

In [5]:
def build_model(input_shape=(28, 28, 1), num_classes=10):
    inputs = layers.Input(shape=input_shape)
    
    # 1. Initial Convolution Layer
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    
    # 2. Global Average Pooling
    gap_features = layers.GlobalAveragePooling2D()(x)
    
    # Reshape and expand to match original spatial dimensions
    gap_features = layers.Dense(64)(gap_features)  # Compress to 64 channels
    gap_features = layers.Reshape((1, 1, 64))(gap_features)
    gap_features = layers.UpSampling2D(size=(28, 28))(gap_features)
    
    # 3. Multiscale Feature Learning
    multiscale_features = multiscale_feature_learning(gap_features)
    
    # 4. Attention Mechanism
    attention_output = AttentionModule()(multiscale_features)
    
    # Final classification head
    x = layers.GlobalAveragePooling2D()(attention_output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    return models.Model(inputs, outputs)

In [6]:
model = build_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 28, 28,    │        640 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 28, 28,    │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,160 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1, 64)  │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 28, 28,    │          0 │ reshape[0][0]     │
│ (UpSampling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 28, 28,    │      2,080 │ up_sampling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 28, 28,    │     18,464 │ up_sampling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 28, 28,    │     51,232 │ up_sampling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 28, 28,    │          0 │ conv2d_1[0][0],   │
│ (Concatenate)       │ 96)               │            │ conv2d_2[0][0],   │
│                     │                   │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_module    │ (None, 28, 28,    │     11,640 │ concatenate[0][0] │
│ (AttentionModule)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 96)        │          0 │ attention_module… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     12,416 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 10)        │      1,290 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 102,178 (399.13 KB)

 Trainable params: 102,050 (398.63 KB)

 Non-trainable params: 128 (512.00 B)

In [9]:
history = model.fit(
    x_train, y_train,
    batch_size=64,
    epochs=2,
    validation_split=0.2,
)

Epoch 1/2
750/750 ━━━━━━━━━━━━━━━━━━━━ 211s 281ms/step - accuracy: 0.4621 - loss: 1.4418 - val_accuracy: 0.3392 - val_loss: 2.7250
Epoch 2/2
750/750 ━━━━━━━━━━━━━━━━━━━━ 207s 275ms/step - accuracy: 0.7638 - loss: 0.7044 - val_accuracy: 0.6552 - val_loss: 1.0359


In [10]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"\nTest accuracy: {test_accuracy:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 41ms/step - accuracy: 0.6347 - loss: 1.0796

Test accuracy: 0.6497
